In [2]:
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

/home/kevin/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_json('./input/train.json')
test = pd.read_json("./input/test.json")
combined = pd.concat([train, test])

In [10]:
combined.loc[combined.latitude > 41, 'latitude'] = np.nan
combined.loc[combined.latitude < 40.5, 'latitude'] = np.nan
combined['lat_scale'] = (combined.latitude - 40.5) * 10000
combined.lat_scale.fillna(-1, inplace=True)

combined.lat_scale.describe()

In [22]:
combined.loc[combined.longitude > -73.74, 'longitude'] = np.nan
combined.loc[combined.longitude < -74.04, 'longitude'] = np.nan
combined['long_scale'] = (combined.longitude + 74.04) * 10000
combined.long_scale.fillna(-1, inplace=True)

combined.long_scale.describe()

In [34]:
train = combined[~pd.isnull(combined.interest_level)]
test = combined[pd.isnull(combined.interest_level)]

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(\
    train[['bathrooms', 'bedrooms','price','lat_scale','long_scale']], \
    train[['interest_level']].values, \
    test_size=0.33, random_state=42)

In [37]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train[:,0])
y_train_encoded = le.transform(y_train[:,0])
y_validate_encoded = le.transform(y_validate[:,0])

In [38]:
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)
dvalidate = xgb.DMatrix(X_validate, label=y_validate_encoded)
evallist  = [(dvalidate,'eval'), (dtrain,'train')]

In [39]:
param = {'bst:max_depth':6, \
         'bst:eta':0.3, \
         'silent':1,
         'objective':'multi:softprob',
         'num_class':3}

num_round = 70
bst = xgb.train(param, dtrain, num_round, evallist )

[0]	eval-merror:0.301652	train-merror:0.289853
[1]	eval-merror:0.29895	train-merror:0.286194
[2]	eval-merror:0.298704	train-merror:0.285438
[3]	eval-merror:0.297538	train-merror:0.284954
[4]	eval-merror:0.29809	train-merror:0.284893
[5]	eval-merror:0.298152	train-merror:0.284591
[6]	eval-merror:0.296187	train-merror:0.283472
[7]	eval-merror:0.295512	train-merror:0.282111
[8]	eval-merror:0.295328	train-merror:0.281688
[9]	eval-merror:0.295082	train-merror:0.280417
[10]	eval-merror:0.294898	train-merror:0.279238
[11]	eval-merror:0.294161	train-merror:0.277968
[12]	eval-merror:0.294161	train-merror:0.277272
[13]	eval-merror:0.294038	train-merror:0.276728
[14]	eval-merror:0.292503	train-merror:0.275125
[15]	eval-merror:0.292442	train-merror:0.274883
[16]	eval-merror:0.292196	train-merror:0.274792
[17]	eval-merror:0.292319	train-merror:0.273703
[18]	eval-merror:0.292442	train-merror:0.272252
[19]	eval-merror:0.291889	train-merror:0.2721
[20]	eval-merror:0.291766	train-merror:0.271405
[21]	e

In [41]:
predict_validate = bst.predict(dvalidate)
from sklearn.metrics import log_loss
print(log_loss(y_validate, predict_validate))

# No better than base line

0.65868458429


In [ ]:
dtest = xgb.DMatrix(test[['bathrooms', 'bedrooms','price','latitude','longitude']])
test_predict = bst.predict(dtest)
test_predict

In [ ]:
result_frame = pd.DataFrame(test_predict)
result_frame.columns = le.classes_
result_frame['listing_id'] = test.listing_id.reset_index().listing_id
result_frame

In [ ]:
# Not submitted
result_frame[['listing_id','high','medium','low']].to_csv('XGB-result.csv', \
                                                          index = False)